# Loan Completion Analysis
## by Jeff Mitchell

## Investigation Overview

The purpose of this analysis is to see what factors impact on the likelihood of a loan being completed. The main focus was on Loan completion and specific factors of the member and the loan that influenced loan completion.

## Dataset Overview

The dataset contains information on nearly 114,000 loans from Prosper. Only loans that had been conclcuded (Paid off succesfully or failed to be paid off) were used for the analysis, which reduced the sample number to 55076. There were loans that were missing values that could not be corrected which resulted in a further 591 loans being removed.

Attributes of interest included Loan completion status, BorrowerAPR, Debt-to-income ratio, Term, Loan Origination Quarter, Monthly Loan Payments and Employment status. There were 81 data points per loan, many of which were not relevant for this analysis. This number was reduced to 16 for the intial analysis.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [ ]:
# load in the dataset into a pandas dataframe
loans = pd.read_csv('prosperLoanData.csv')

## (Visualization 1)

> Write a comment about your visualization here. The visualization should be in
the next cell, as a sub-slide type. Make sure your visualizations are polished!

## (Visualization 2)

> You should have at least three visualizations in your presentation,
but feel free to add more if you'd like!

## (Visualization 3)



> Once you're ready to finish your presentation, check your output by using
nbconvert to export the notebook and set up a server for the slides. From the
terminal or command line, use the following expression:
> > `jupyter nbconvert <file_name>.ipynb --to slides --post serve --template output_toggle`

> This should open a tab in your web browser where you can scroll through your
presentation. Sub-slides can be accessed by pressing 'down' when viewing its parent
slide. Make sure you remove all of the quote-formatted guide notes like this one
before you finish your presentation!